# Generating gains for wav calibration on augmented soundscapes

In ARAUS dataset responses.csv constitute the dataset of +25k augmented soundscapes labeled with psychoacoustic and acoustic parametres. Among these, we can find Leq_r, which constitutes the Leq of channel R for each audio. These value is calculated according to the formula below. We calculate the Leq on the digital signal channel R, without any wav calibration. The obtained value is sustracted from the corresponding Leq_r. The difference is converted to lineal, and it constitutes the wav calibration we need to apply to the audio.

In [3]:
# Add MOSQITO to the Python path
import sys
import os
sys.path.append('..')

# Import numpy
import numpy as np
import pandas as pd
# Import plot function
import matplotlib.pyplot as plt
# Import mosqito functions
#from mosqito.utils import load
from scipy.io import wavfile
from scipy.signal import resample
from maad.spl import pressure2leq, pressure2dBSPL
from maad.util import mean_dB
import librosa

# Import MOSQITO color sheme [Optional]
from mosqito import COLORS

In [4]:
def load(file, wav_calib=None, mat_signal="", mat_fs="", ch=None):
    """Extract the signal and its time axis from .wav or .uff file,
    resample the signal to 48 kHz, and affects its sampling frequency
    and time signal values.

    Parameters
    ----------
    file : string
        string path to the signal file
    wav_calib : float, optional
        Wav file calibration factor [Pa/FS]. Level of the signal in Pa_peak
        corresponding to the full scale of the .wav file. If None, a
        calibration factor of 1 is considered. Default to None.
    mat_signal : string
        in case of a .mat file, name of the signal variable
    mat_fs : string
        in case of a .mat file, name of the sampling frequency variable

    Outputs
    -------
    signal : numpy.array
        time signal values
    fs : integer
        sampling frequency
    """

    # load the .wav file content
    if file[-3:] == "wav" or file[-3:] == "WAV":
        fs, signal = wavfile.read(file)

        # manage multichannel files
        if signal.ndim > 1:
            signal = signal[:, ch] # signal[:, 0] for first channel, signal[:, 1] for second ch

        # calibration factor for the signal to be in Pa
        if wav_calib is None:
            wav_calib = 1
            print("[Info] A calibration of 1 Pa/FS is considered")
        if isinstance(signal[0], np.int16):
            signal = wav_calib * signal / (2**15 - 1)
        elif isinstance(signal[0], np.int32):
            signal = wav_calib * signal / (2**31 - 1)
        elif isinstance(signal[0], np.float):
            signal = wav_calib * signal

    else:
        raise ValueError("""ERROR: only .wav .mat or .uff files are supported""")

    # resample to 48kHz to allow calculation
    if fs != 48000:
        signal = resample(signal, int(48000 * len(signal) / fs))
        fs = 48000

    return signal, fs


In [20]:
data_augmented = pd.read_csv("../data/responses_complete.csv")
# For each audio...
audios_path = "../data/soundscapes_augmented/"
n_audios=data_augmented.shape[0]
gain_values=np.zeros(n_audios)
i=0
for file in sorted(os.listdir(audios_path)):
        if file.endswith('.mp3') or file.endswith('.wav'):
            print("file name ", file)
            # locate audio in dataset
            file_split=file.split("_")
            file_fold=int(file_split[1])
            file_participant="ARAUS_"+file_split[3]
            file_stimulus=int(file_split[5].split(".")[0])
            audio_info_aug = data_augmented[
                (data_augmented['fold_r'] == file_fold) &
                (data_augmented['stimulus_index'] == file_stimulus) &
                (data_augmented['participant'] == file_participant)
            ]
            row_index = audio_info_aug.index[0]
            print(row_index)
            """ audio_info_aug = audio_info_aug[audio_info_aug['stimulus_index'] == file_stimulus]
            audio_info_aug = audio_info_aug[audio_info_aug['participant'] == file_participant] # Row of info from responses.csv """
            # audio path
            audio_path=audios_path+file
            #Get Leq of raw digital audio to calculate audio gain needed
            rawR, fs = load(audio_path, wav_calib=1, ch=1) #R
            rawR_Leq_avg=mean_dB(pressure2leq(rawR, fs, 0.125))
            trueLeq=data_augmented["Leq_R_r"].values[0]
            difference=trueLeq-rawR_Leq_avg
            gain_values[row_index]=10**(difference/20)
            i=i+1
            print("Done audio number ", i, "/",n_audios)


file name  fold_0_participant_10001_stimulus_02.wav
27001
Done audio number  1 / 27255
file name  fold_0_participant_10001_stimulus_03.wav
27002
Done audio number  2 / 27255
file name  fold_0_participant_10001_stimulus_04.wav
27003
Done audio number  3 / 27255
file name  fold_0_participant_10001_stimulus_05.wav
27004
Done audio number  4 / 27255
file name  fold_0_participant_10001_stimulus_06.wav
27005
Done audio number  5 / 27255
file name  fold_0_participant_10001_stimulus_07.wav
27006
Done audio number  6 / 27255
file name  fold_0_participant_10001_stimulus_08.wav
27007
Done audio number  7 / 27255
file name  fold_0_participant_10001_stimulus_09.wav
27008
Done audio number  8 / 27255
file name  fold_0_participant_10001_stimulus_10.wav
27009
Done audio number  9 / 27255
file name  fold_0_participant_10001_stimulus_11.wav
27010
Done audio number  10 / 27255
file name  fold_0_participant_10001_stimulus_12.wav
27011
Done audio number  11 / 27255
file name  fold_0_participant_10001_stimu

In [26]:

# Create a new column with the generated values
data_augmented_copy=data_augmented.copy(deep=True)
data_augmented_copy.insert(loc=6, column='wav_gain', value=gain_values)


In [28]:
print(data_augmented_copy.columns)

Index(['participant', 'fold_r', 'soundscape', 'masker', 'smr',
       'stimulus_index', 'wav_gain', 'time_taken', 'is_attention', 'pleasant',
       ...
       'Leq_L_r', 'Leq_R_r', 'masker_bird', 'masker_construction',
       'masker_silence', 'masker_traffic', 'masker_water', 'masker_wind',
       'P_ground_truth', 'E_ground_truth'],
      dtype='object', length=169)


In [29]:
# Save new dataset

data_augmented_copy.to_csv("../data/responses_wav_gains.csv", index=False)